In [1]:
# coding: utf-8

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt    #グラフ出力用module
from PIL import Image
from sklearn.model_selection import train_test_split
import glob
import os
import csv

#BATCH_SIZE = 100
#WEIGHT_DECAY = 0.00005
#LEARNING_RATE = 0.0001
#EPOCH = 100
PATH = os.getcwd()
#TRAINPATH = "C:\\Users\\arimoto\\cnntest\\train"
#TESTPATH = "C:\\Users\\arimoto\\cnntest\\test"
#VALIDPATH = "C:\\Users\\arimoto\\cnntest\\valid"

#手指姿勢CSVファイル一覧取得
datafilelists = glob.glob(PATH + "\\dataset\\data\\*.csv")
labelfilelists = glob.glob(PATH + "\\dataset\\label\\*.csv")
#print(filelists)

data_sets = []
label_sets = []
valid_data_num = 0
invalid_data_num = 0
valid_label_num = 0
invalid_label_num = 0
for i in range(len(datafilelists)):
    data_points = []
    data_masks = []
    file_name = datafilelists[i]
    datenum = int(file_name[-33:-25] + file_name[-24:-18] + file_name[-14:-9])
    #print(datenum)
    invcnt = 0
    with open(datafilelists[i]) as f:
        reader = csv.reader(f)
        num = 0
        for row in reader:
            if num == 0:
                for point in row:
                    if float(point) == -10000:
                        invcnt += 1
                        data_points.append(0)
                        data_masks.append(0)
                    else:
                        data_points.append(float(point))
                        data_masks.append(1)
                data_points = np.asarray(data_points)
                data_masks = np.asarray(data_masks)
                num += 1
    if invcnt <= 6:
        data_pair = []
        data_pair.append(data_points)
        data_pair.append(data_masks)
        data_pair.append(datenum)
        data_sets.append(data_pair)
        valid_data_num += 1
    else:
        invalid_data_num += 1
for i in range(len(labelfilelists)):
    label_points = []
    label_masks = []
    file_name = labelfilelists[i]
    datenum = int(file_name[-33:-25] + file_name[-24:-18] + file_name[-14:-9])
    #print(datenum)
    invcnt = 0
    with open(labelfilelists[i]) as f:
        reader = csv.reader(f)
        num = 0
        for row in reader:
            if num == 0:
                for point in row:
                    if float(point) == -10000:
                        invcnt += 1
                        label_points.append(0)
                        label_masks.append(0)
                    else:
                        label_points.append(float(point))
                        label_masks.append(1)
                label_points = np.asarray(label_points)
                label_masks = np.asarray(label_masks)
                num += 1
    if invcnt <= 6:
        label_pair = []
        label_pair.append(label_points)
        label_pair.append(label_masks)
        label_pair.append(datenum)
        label_sets.append(label_pair)
        valid_label_num += 1
    else:
        invalid_label_num += 1        


print("fin loading")

fin loading


In [2]:
data_label_sets = []
for i in range(len(data_sets)):
    data_label_allset = []
    data_label_allset.append(data_sets[i][0])
    data_label_allset.append(data_sets[i][1])
    data_label_allset.append(data_sets[i][2])
    data_label_allset.append(label_sets[i][0])
    data_label_allset.append(label_sets[i][1])
    data_label_sets.append(data_label_allset)
#data_label_sets.append(data_sets)
#data_label_sets.append(label_sets)
data_label_sets = np.asarray(data_label_sets)
#print("data" + str(data_sets[0]))
#print("data", data_label_sets[0][0])
#print("label" + str(label_sets[0]))
#print("label", data_label_sets[1][0])
#print(type(data_sets[0][0]))
#print(type(label_sets[0][0]))
print("data_label_sets", data_label_sets.shape)
print("valid data: ", valid_data_num)
print("invalid data: ", invalid_data_num)
print("valid label: ", valid_label_num)
print("invalid label: ", invalid_label_num)

data_label_sets (5690, 5)
valid data:  5690
invalid data:  11
valid label:  5690
invalid label:  11


In [3]:
# 数合わせ用に使わないデータを切り捨て
data_label_sets, govage_sets = train_test_split(
    data_label_sets,
    shuffle = False,
    train_size = 5600
)
#label_sets, govage_sets = train_test_split(
#    label_sets,
#    shuffle = False,
#    train_size = 5600
#)
print(data_label_sets.shape, govage_sets.shape)
print(data_label_sets[0])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(5600, 5) (90, 5)
[array([   0.      ,    0.      ,    0.      ,    0.593171,   10.7672  ,
        -23.7559  ,   15.      ,    0.645983,   16.2755  ,  -51.675   ,
         48.      ,    0.836505,   12.7484  ,  -72.6442  ,   67.      ,
          0.863634,   -6.66014 ,  -84.6189  ,   77.1766  ,    0.782464,
        -34.0331  ,  -66.2321  ,    6.      ,    0.799855,  -47.8311  ,
        -87.8505  ,   63.      ,    0.811137,  -62.3664  ,  -97.5839  ,
         71.      ,    0.846868,  -75.3266  , -103.475   ,   76.      ,
          0.834375,  -45.2463  ,  -53.5921  ,   48.      ,    0.799601,
        -68.3059  ,  -78.4652  ,   67.      ,    0.691475,  -91.1309  ,
        -89.8903  ,   66.      ,    0.871703, -106.322   ,  -95.6192  ,
         66.      ,    0.865382,  -56.6348  ,  -40.4003  ,   46.      ,
          0.802496,  -81.7476  ,  -61.3173  ,   64.      ,    0.877817,
       -102.496   ,  -68.9664  ,   66.      ,    0.758722, -119.601   ,
        -74.6921  ,   66.      ,    0.861539,

In [4]:
# trainデータとtestデータに分割
data_label_sets_train, data_label_sets_test = train_test_split(
    data_label_sets,
    shuffle = True,
    random_state = 0,
    test_size = 0.2
)
print(data_label_sets_train.shape, data_label_sets_test.shape)
print(data_label_sets_train[0])
print(data_label_sets_train[3])

(4480, 5) (1120, 5)
[array([ 0.00000e+00,  0.00000e+00,  0.00000e+00,  3.71683e-01,
        6.94178e+00, -1.16422e+01,  1.00000e+00,  2.79978e-01,
        3.97565e+00, -2.05407e+01,  1.00000e+01,  1.95300e-01,
        5.31496e-01, -1.88305e+01,  1.00000e+01,  1.10865e-01,
       -1.34814e+01,  7.81400e-01,  1.00000e+01,  6.87725e-02,
       -4.01789e+01, -6.28342e+00,  3.90000e+01,  5.45225e-01,
       -6.43678e+01,  1.11345e+01,  7.70000e+01,  6.09153e-01,
       -8.22194e+01,  2.26143e+01,  7.70000e+01,  7.79267e-01,
       -1.02373e+02,  3.33824e+01,  7.70000e+01,  7.74360e-01,
       -4.88439e+01,  6.57513e+00,  4.50000e+01,  5.70897e-01,
       -7.68159e+01,  3.24046e+01,  9.10000e+01,  5.56875e-01,
       -9.49595e+01,  4.78879e+01,  9.10000e+01,  4.81774e-01,
       -1.09880e+02,  6.28536e+01,  9.10000e+01,  4.58244e-01,
       -5.21422e+01,  2.28144e+01,  6.20000e+01,  7.91982e-01,
       -7.68178e+01,  3.92391e+01,  9.00000e+01,  8.14949e-01,
       -9.31363e+01,  5.32041e+01,

In [5]:
data_train = []
mask_train = []
adrs_train = []
data_test = []
mask_test = []
adrs_test = []
label_train = []
lmask_train = []
ladrs_train = []
label_test = []
lmask_test = []
ladrs_test = []
for i in range(len(data_label_sets_train)):
    data_train.append(data_label_sets_train[i][0])
    mask_train.append(data_label_sets_train[i][1])
    adrs_train.append(data_label_sets_train[i][2])
    label_train.append(data_label_sets_train[i][3])
    lmask_train.append(data_label_sets_train[i][4])
    ladrs_train.append(data_label_sets_train[i][2])
for i in range(len(data_label_sets_test)):
    data_test.append(data_label_sets_test[i][0])
    mask_test.append(data_label_sets_test[i][1])
    adrs_test.append(data_label_sets_test[i][2])
    label_test.append(data_label_sets_test[i][3])
    lmask_test.append(data_label_sets_test[i][4])
    ladrs_test.append(data_label_sets_test[i][2])

print(data_train[0])
print(mask_train[0])
print(adrs_train[0])
print(label_train[0])
print(lmask_train[0])
print(ladrs_train[0])

[ 0.00000e+00  0.00000e+00  0.00000e+00  3.71683e-01  6.94178e+00
 -1.16422e+01  1.00000e+00  2.79978e-01  3.97565e+00 -2.05407e+01
  1.00000e+01  1.95300e-01  5.31496e-01 -1.88305e+01  1.00000e+01
  1.10865e-01 -1.34814e+01  7.81400e-01  1.00000e+01  6.87725e-02
 -4.01789e+01 -6.28342e+00  3.90000e+01  5.45225e-01 -6.43678e+01
  1.11345e+01  7.70000e+01  6.09153e-01 -8.22194e+01  2.26143e+01
  7.70000e+01  7.79267e-01 -1.02373e+02  3.33824e+01  7.70000e+01
  7.74360e-01 -4.88439e+01  6.57513e+00  4.50000e+01  5.70897e-01
 -7.68159e+01  3.24046e+01  9.10000e+01  5.56875e-01 -9.49595e+01
  4.78879e+01  9.10000e+01  4.81774e-01 -1.09880e+02  6.28536e+01
  9.10000e+01  4.58244e-01 -5.21422e+01  2.28144e+01  6.20000e+01
  7.91982e-01 -7.68178e+01  3.92391e+01  9.00000e+01  8.14949e-01
 -9.31363e+01  5.32041e+01  9.00000e+01  7.39952e-01 -1.09880e+02
  6.28536e+01  9.00000e+01  7.08342e-01 -5.21712e+01  3.65935e+01
  6.20000e+01  7.76721e-01 -6.61830e+01  4.76072e+01  7.70000e+01
  7.50923e

In [6]:
data_trainF = np.array(data_train).astype('float32')/100
data_testF = np.array(data_test).astype('float32')/100
mask_trainF = np.array(mask_train).astype('float32')
mask_testF = np.array(mask_test).astype('float32')
adrs_trainF = np.array(adrs_train)
adrs_testF = np.array(adrs_test)
label_trainF = np.array(label_train).astype('float32')/100
label_testF = np.array(label_test).astype('float32')/100
lmask_trainF = np.array(lmask_train).astype('float32')
lmask_testF = np.array(lmask_test).astype('float32')
ladrs_trainF = np.array(ladrs_train)
ladrs_testF = np.array(ladrs_test)

adrs_trainF = adrs_trainF[:, np.newaxis]
ladrs_trainF = ladrs_trainF[:, np.newaxis]
adrs_testF = adrs_testF[:, np.newaxis]
ladrs_testF = ladrs_testF[:, np.newaxis]

print("data          train ",data_trainF.shape)
print("data          test  ",data_testF.shape)
print("data-mask     train ",mask_trainF.shape)
print("data-mask     test  ",mask_testF.shape)
print("address       train ",adrs_trainF.shape)
print("address       test  ",adrs_testF.shape)
print("label         train ",label_trainF.shape)
print("label         test  ",label_testF.shape)
print("label-mask    train ",lmask_trainF.shape)
print("label-mask    test  ",lmask_testF.shape)
print("label-address train ",ladrs_trainF.shape)
print("label-address test  ",ladrs_testF.shape)

data          train  (4480, 84)
data          test   (1120, 84)
data-mask     train  (4480, 84)
data-mask     test   (1120, 84)
address       train  (4480, 1)
address       test   (1120, 1)
label         train  (4480, 63)
label         test   (1120, 63)
label-mask    train  (4480, 63)
label-mask    test   (1120, 63)
label-address train  (4480, 1)
label-address test   (1120, 1)


In [7]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, datas, labels, data_masks, label_masks, adrs, label_adrs, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = datas
        self.label = labels
        self.masks = data_masks
        self.lmasks = label_masks
        self.adrs = adrs
        self.ladrs = label_adrs
        
        self.datanum = datas.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        #batch_set = []
        
        i_data = self.data[idx]
        i_label = self.label[idx]
        i_mask = self.masks[idx]
        i_lmask = self.lmasks[idx]
        i_adrs = self.adrs[idx]
        i_ladrs = self.ladrs[idx]
        
        #print(out_label)
        #print(type(i_label))
        out_label = np.array(i_label.astype(np.float32))
        out_data = np.array(i_data.astype(np.float32))
        out_mask = np.array(i_mask.astype(np.float32))
        out_lmask = np.array(i_lmask.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        #batch_set.append(out_data)
        #batch_set.append(out_label)
        #batch_set.append(out_img)
        #batch_set.append(out_mask)
        #batch_set.append(out_lmask)
        #print(out_data.shape)
        #print(out_label.shape)
        #print(out_mask.shape)
        #print(out_lmask.shape)
        
        return out_data, out_label, out_mask, out_lmask, i_adrs, i_ladrs
        #return batch_set

In [8]:
trainset = Mydatasets(datas = data_trainF,
                      labels = label_trainF,
                      data_masks = mask_trainF,
                      label_masks = lmask_trainF,
                      adrs = adrs_trainF,
                      label_adrs = ladrs_trainF
                     )
testset = Mydatasets(datas = data_testF,
                     labels = label_testF,
                     data_masks = mask_testF,
                     label_masks = lmask_testF,
                     adrs = adrs_testF,
                     label_adrs = ladrs_testF
                    )

In [9]:
BATCH_SIZE = 112
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE,
                        shuffle = True, num_workers = 0)
testloader = torch.utils.data.DataLoader(testset, batch_size = BATCH_SIZE,
                        shuffle = False, num_workers = 0)

In [10]:
class Ignore(nn.Module):
    def __init__(self, input_dim, output_dim, activation = lambda x: x):
        '''
        引数:
            input_dim: 入力次元
            output_dim: 出力次元
            activation: 活性化関数
        パラメータ:
            W: 重み
            b: バイアス
        '''
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(np.random.normal(size = (output_dim, input_dim))))
        self.b = nn.Parameter(torch.Tensor(np.zeros(output_dim)))
        self.activation = activation
    def forward(self, x, mask):
        
        out = torch.empty_like(x).to(torch.device("cuda:0"))
        
        masked_x = torch.mul(x, mask)
        
        try:
            m_size = torch.Tensor(mask.size()[1]).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask, 1).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            for b in range(out.size()[0]):
                out[b] = torch.add(torch.mul(rate[b], torch.sum(torch.mul(masked_x[b], self.W), 1)), self.b)
        except IndexError:
            m_size = torch.Tensor(mask.size()).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            out = torch.add(torch.mul(rate, torch.sum(torch.mul(masked_x, self.W))), self.b)
        
        return self.activation(out)

In [19]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        #CNN
        self.pool = nn.AvgPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)
        self.bn2d1 = nn.BatchNorm2d(16)
        self.bn2d2 = nn.BatchNorm2d(32)
        
        self.fcm = Ignore(32 * 7 * 7 + 64, 32 * 7 * 7 + 64)
        self.bnm = nn.BatchNorm1d(32 * 7 * 7 + 64)
        #fully connect for hand Location(x,y,z)
        self.fcL1 = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcL2 = nn.Linear(300, 30)
        self.fcL3 = nn.Linear(30, 3)        
        self.bnL1 = nn.BatchNorm1d(300)
        self.bnL2 = nn.BatchNorm1d(30)
        #fully connect for hand Pose Descriptor(8 properties)
        self.fcPD1 = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcPD2 = nn.Linear(300, 60)
        self.fcPD3 = nn.Linear(60, 8)
        self.bnPD1 = nn.BatchNorm1d(300)
        self.bnPD2 = nn.BatchNorm1d(60)
        
        #Autoencoder
        self.mask = Ignore(84, 84)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.dense_enc1 = nn.Linear(84, 40)
        self.bn1 = nn.BatchNorm1d(40)
        self.dense_enc2 = nn.Linear(40, 18)
        self.bn2 = nn.BatchNorm1d(18)
        self.dense_enc3 = nn.Linear(18,8)
    
        self.dense_dec1 = nn.Linear(8,16)
        self.bn4 = nn.BatchNorm1d(16)
        self.dense_dec2 = nn.Linear(16, 32)
        self.bn5 = nn.BatchNorm1d(32)
        self.drop1 = nn.Dropout(p=0.2)
        self.dense_dec3 = nn.Linear(32, 63)

    def encoder(self, x):
        #x = torch.div(x, 100.)
        #x = self.mask(x, m)
        x = self.dense_enc1(x)
        x = self.bn1(self.relu(x))
        x = self.dense_enc2(x)
        x = self.bn2(self.relu(x))
        x = self.dense_enc3(x)
        return x

    def decoder(self, x):
        x = self.dense_dec1(x)
        x = self.bn4(self.relu(x))
        x = self.dense_dec2(x)
        x = self.bn5(self.relu(x))
        #x = self.drop1(x)
        x = self.dense_dec3(x)
        #x = self.mask(x, m)
        #x = torch.mul(x, 100.)
        return x

    def forward(self, x=None, z=None):
        if x != None and z == None:
            z = self.encoder(x)
            x = self.decoder(z)
        elif x == None and z != None:
            print("decoder only")
            x = self.decoder(z)
        return x, z

In [20]:
class MaskMSELoss(nn.Module):

    def __init__(self):
        super(MaskMSELoss, self).__init__()
        #self.margin = margin

    def forward(self, inputs, labels, lmasks):
        m_size = lmasks.size()[1]
        m_sum = torch.sum(lmasks, 1)
        rate = (m_size / m_sum).reshape(BATCH_SIZE,-1)
        for i in range(rate.size()[0]):
            if np.isinf(rate[i].item()):
                rate[i] = 0.
        masked_in = torch.mul(inputs, lmasks)
        masked_lb = torch.mul(labels, lmasks)
        
        #loss = torch.sub(masked_in, masked_lb)
        #loss = torch.pow(loss, 2)
        #loss = torch.sum(loss, 1)
        #loss = torch.mul(rate, loss)
        #loss = torch.mean(loss)
        
        loss = torch.sub(masked_in, masked_lb)
        loss = torch.pow(loss, 2)
        loss = torch.mul(loss, rate)
        loss = torch.mean(loss, 1)
        loss = torch.mean(loss)
        
        return loss

In [24]:
LEARNING_RATE = 0.01
WEIGHT_DECAY = 0.0001
MOMENTUM = 0.9
#EPOCH = 100
device = torch.device("cuda:0")
net = Autoencoder()
net = net.to(device)
criterion = MaskMSELoss()
#criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

train_loss_value=[]      #trainingのlossを保持するlist
test_loss_value=[]       #testのlossを保持するlist

train_mid_out_value = [] #trainingの中間層の出力を保持するlist
test_mid_out_value = []  #testの中間層の出力を保持するlist
train_input_value = []   #trainingの入力を保持するlist
test_input_value = []    #testの入力を保持するlist
train_output_value = []  #trainingの出力を保持するlist
test_output_value = []   #testの出力を保持するlist
train_adrs = []
test_adrs = []

train_size = data_trainF.shape[0]
test_size = data_testF.shape[0] 

max_train_loss_value = 0.
max_test_loss_value = 0.

In [25]:
EPOCH = 1000
for epoch in range(EPOCH):#EPOCH):
    print('epoch', epoch+1)    #epoch数の出力
    num = 0
    for (inputs, labels, input_mask, label_mask, adrs, ladrs) in trainloader:
        #num += 1
        #print(num)
        inputs, labels, input_mask, label_mask = \
        inputs.to(device), labels.to(device), input_mask.to(device), label_mask.to(device)
        optimizer.zero_grad()
        #print("input(train): ", inputs.size())
        outputs, middle = net(inputs)#, input_mask)
        #print("input(train): ", inputs[0], inputs[1])
        #print("middle(train): ", middle[0], middle[1])
        #print("output(train): ", outputs[0], outputs[1])
        loss = criterion(outputs, labels, label_mask)#crit_outputs, crit_labels)
        ##loss = criterion(outputs, labels)
        ##print("loss(train): ", loss)
        loss.backward()
        optimizer.step()

    sum_loss = 0.0          #lossの合計
    #sum_correct = 0         #正解率の合計
    sum_total = 0           #dataの数の合計

    print("test_train")
    ttrain = 0
    #train dataを使ってテストをする(パラメータ更新がないようになっている)
    for (inputs, labels, input_mask, label_mask, adrs, ladrs) in trainloader:
        ttrain += 1
        inputs, labels, input_mask, label_mask = \
        inputs.to(device), labels.to(device), input_mask.to(device), label_mask.to(device)
        optimizer.zero_grad()
        outputs, middle = net(inputs)#, input_mask)
        if ttrain == int(train_size / BATCH_SIZE):
            ##print("input(train_test): ", inputs[0], inputs[1])
            ##print("middle(train_test): ", middle[0], middle[1])
            ##print("output(train_test): ", outputs[0], outputs[1])
            train_input_value.append(inputs)
            train_mid_out_value.append(middle)
            train_output_value.append(outputs)
            #print(adrs)
            #print(ladrs)
            train_adrs.append(adrs)
        loss = criterion(outputs*100, labels*100, label_mask)
        ##loss = criterion(outputs, labels)
        ##print("loss(train_test): ", loss)
        sum_loss += loss.item()                            #lossを足していく
        #_, predicted = outputs.max(1)                      #出力の最大値の添字(予想位置)を取得
        sum_total += labels.size(0)                        #labelの数を足していくことでデータの総和を取る
        #sum_correct += (predicted == labels).sum().item()  #予想位置と実際の正解を比べ,正解している数だけ足す
        
    print("len train dataset: ", len(trainloader.dataset))
    print("train mean loss={}".format(sum_loss*BATCH_SIZE/len(trainloader.dataset)))  #loss出力
    train_loss_value.append(sum_loss*BATCH_SIZE/len(trainloader.dataset))  #traindataのlossをグラフ描画のためにlistに保持
    #train_acc_value.append(float(sum_correct/sum_total))   #traindataのaccuracyをグラフ描画のためにlistに保持
    if sum_loss*BATCH_SIZE/len(trainloader.dataset) > max_train_loss_value:
        max_train_loss_value = sum_loss*BATCH_SIZE/len(trainloader.dataset)

    sum_loss = 0.0
    #sum_correct = 0
    sum_total = 0

    print("test_test")
    ttest = 0
    #test dataを使ってテストをする
    for (inputs, labels, input_mask, label_mask, adrs, ladrs) in testloader:
        ttest += 1
        inputs, labels, input_mask, label_mask = \
        inputs.to(device), labels.to(device), input_mask.to(device), label_mask.to(device)
        optimizer.zero_grad()
        outputs, middle = net(inputs)#, input_mask)
        if ttest == int(test_size / BATCH_SIZE):
            ##print("input(test_test): ", inputs[0], inputs[1])
            ##print("middle(test_test): ", middle[0], middle[1])
            ##print("output(test_test): ", outputs[0], outputs[1])
            test_input_value.append(inputs)
            test_mid_out_value.append(middle)
            test_output_value.append(outputs)
            #print(adrs)
            #print(ladrs)
            test_adrs.append(adrs)
        loss = criterion(outputs*100, labels*100, label_mask)
        ##loss = criterion(outputs, labels)
        ##print("loss(test_test): ", loss)
        sum_loss += loss.item()
        #_, predicted = outputs.max(1)
        sum_total += labels.size(0)
        #sum_correct += (predicted == labels).sum().item()
    print("len test dataset: ", len(testloader.dataset))
    print("test mean loss={}"
            .format(sum_loss*BATCH_SIZE/len(testloader.dataset)))
    test_loss_value.append(sum_loss*BATCH_SIZE/len(testloader.dataset))
    #test_acc_value.append(float(sum_correct/sum_total))
    if sum_loss*BATCH_SIZE/len(testloader.dataset) > max_test_loss_value:
        max_test_loss_value = sum_loss*BATCH_SIZE/len(testloader.dataset)

epoch 1
test_train
len train dataset:  4480
train mean loss=3043.562536621094
test_test
len test dataset:  1120
test mean loss=2685.2703369140627
epoch 2
test_train
len train dataset:  4480
train mean loss=1744.2126068115235
test_test
len test dataset:  1120
test mean loss=1446.3812377929687
epoch 3
test_train
len train dataset:  4480
train mean loss=1315.6067474365234
test_test
len test dataset:  1120
test mean loss=1097.7308654785156
epoch 4
test_train
len train dataset:  4480
train mean loss=1055.282341003418
test_test
len test dataset:  1120
test mean loss=922.1681945800781
epoch 5
test_train
len train dataset:  4480
train mean loss=897.647737121582
test_test
len test dataset:  1120
test mean loss=801.1733276367188
epoch 6
test_train
len train dataset:  4480
train mean loss=819.1395309448242
test_test
len test dataset:  1120
test mean loss=735.049658203125
epoch 7
test_train
len train dataset:  4480
train mean loss=757.7633636474609
test_test
len test dataset:  1120
test mean loss=

test mean loss=264.3583068847656
epoch 57
test_train
len train dataset:  4480
train mean loss=331.9597171783447
test_test
len test dataset:  1120
test mean loss=268.0288513183594
epoch 58
test_train
len train dataset:  4480
train mean loss=331.1668872833252
test_test
len test dataset:  1120
test mean loss=262.2648300170898
epoch 59
test_train
len train dataset:  4480
train mean loss=323.73366355895996
test_test
len test dataset:  1120
test mean loss=260.47630004882814
epoch 60
test_train
len train dataset:  4480
train mean loss=326.03253135681155
test_test
len test dataset:  1120
test mean loss=260.53443603515626
epoch 61
test_train
len train dataset:  4480
train mean loss=318.8533111572266
test_test
len test dataset:  1120
test mean loss=258.6422058105469
epoch 62
test_train
len train dataset:  4480
train mean loss=326.15880584716797
test_test
len test dataset:  1120
test mean loss=257.40497436523435
epoch 63
test_train
len train dataset:  4480
train mean loss=315.7117469787598
test_t

train mean loss=284.14024620056153
test_test
len test dataset:  1120
test mean loss=221.80874938964843
epoch 113
test_train
len train dataset:  4480
train mean loss=279.1433387756348
test_test
len test dataset:  1120
test mean loss=223.56290740966796
epoch 114
test_train
len train dataset:  4480
train mean loss=279.6876335144043
test_test
len test dataset:  1120
test mean loss=225.1651809692383
epoch 115
test_train
len train dataset:  4480
train mean loss=283.75243339538576
test_test
len test dataset:  1120
test mean loss=221.07477111816405
epoch 116
test_train
len train dataset:  4480
train mean loss=278.4141128540039
test_test
len test dataset:  1120
test mean loss=226.33863677978516
epoch 117
test_train
len train dataset:  4480
train mean loss=273.0736446380615
test_test
len test dataset:  1120
test mean loss=221.3259704589844
epoch 118
test_train
len train dataset:  4480
train mean loss=272.2707328796387
test_test
len test dataset:  1120
test mean loss=218.33832550048828
epoch 119


len test dataset:  1120
test mean loss=204.45142211914063
epoch 168
test_train
len train dataset:  4480
train mean loss=253.7825839996338
test_test
len test dataset:  1120
test mean loss=204.67779693603515
epoch 169
test_train
len train dataset:  4480
train mean loss=246.44755859375
test_test
len test dataset:  1120
test mean loss=202.49278411865234
epoch 170
test_train
len train dataset:  4480
train mean loss=252.1599422454834
test_test
len test dataset:  1120
test mean loss=205.21702575683594
epoch 171
test_train
len train dataset:  4480
train mean loss=250.12457962036132
test_test
len test dataset:  1120
test mean loss=201.98979797363282
epoch 172
test_train
len train dataset:  4480
train mean loss=240.60009803771973
test_test
len test dataset:  1120
test mean loss=202.8450958251953
epoch 173
test_train
len train dataset:  4480
train mean loss=248.3343490600586
test_test
len test dataset:  1120
test mean loss=204.12061920166016
epoch 174
test_train
len train dataset:  4480
train mea

test mean loss=188.72823181152344
epoch 223
test_train
len train dataset:  4480
train mean loss=225.99291801452637
test_test
len test dataset:  1120
test mean loss=192.9201675415039
epoch 224
test_train
len train dataset:  4480
train mean loss=227.25441932678223
test_test
len test dataset:  1120
test mean loss=188.48076477050782
epoch 225
test_train
len train dataset:  4480
train mean loss=223.85984077453614
test_test
len test dataset:  1120
test mean loss=188.30937805175782
epoch 226
test_train
len train dataset:  4480
train mean loss=236.4426649093628
test_test
len test dataset:  1120
test mean loss=190.51597900390624
epoch 227
test_train
len train dataset:  4480
train mean loss=228.7729591369629
test_test
len test dataset:  1120
test mean loss=188.52556457519532
epoch 228
test_train
len train dataset:  4480
train mean loss=218.18558101654054
test_test
len test dataset:  1120
test mean loss=187.38171997070313
epoch 229
test_train
len train dataset:  4480
train mean loss=225.696741104

...

test_test
len test dataset:  1120
test mean loss=153.01986541748047
epoch 774
test_train
len train dataset:  4480
train mean loss=170.2606004714966
test_test
len test dataset:  1120
test mean loss=154.014070892334
epoch 775
test_train
len train dataset:  4480
train mean loss=162.70808238983153
test_test
len test dataset:  1120
test mean loss=153.93855361938478
epoch 776
test_train
len train dataset:  4480
train mean loss=171.476544380188
test_test
len test dataset:  1120
test mean loss=152.70709075927735
epoch 777
test_train
len train dataset:  4480
train mean loss=167.36691303253173
test_test
len test dataset:  1120
test mean loss=155.95000534057618
epoch 778
test_train
len train dataset:  4480
train mean loss=173.09481868743896
test_test
len test dataset:  1120
test mean loss=155.11817016601563
epoch 779
test_train
len train dataset:  4480
train mean loss=170.10499725341796
test_test
len test dataset:  1120
test mean loss=153.92030563354493
epoch 780
test_train
len train dataset:  44

test mean loss=154.19964370727538
epoch 829
test_train
len train dataset:  4480
train mean loss=162.46544780731202
test_test
len test dataset:  1120
test mean loss=153.00330047607423
epoch 830
test_train
len train dataset:  4480
train mean loss=165.28754005432128
test_test
len test dataset:  1120
test mean loss=154.4087387084961
epoch 831
test_train
len train dataset:  4480
train mean loss=171.7879795074463
test_test
len test dataset:  1120
test mean loss=154.07758712768555
epoch 832
test_train
len train dataset:  4480
train mean loss=162.89627323150634
test_test
len test dataset:  1120
test mean loss=152.51968841552736
epoch 833
test_train
len train dataset:  4480
train mean loss=170.818106842041
test_test
len test dataset:  1120
test mean loss=152.55788879394532
epoch 834
test_train
len train dataset:  4480
train mean loss=169.5755811691284
test_test
len test dataset:  1120
test mean loss=153.74423370361328
epoch 835
test_train
len train dataset:  4480
train mean loss=164.55542659759

test mean loss=151.5131721496582
epoch 884
test_train
len train dataset:  4480
train mean loss=173.5122356414795
test_test
len test dataset:  1120
test mean loss=153.20176391601564
epoch 885
test_train
len train dataset:  4480
train mean loss=171.19786396026612
test_test
len test dataset:  1120
test mean loss=157.65713272094726
epoch 886
test_train
len train dataset:  4480
train mean loss=168.56484813690184
test_test
len test dataset:  1120
test mean loss=153.07293930053712
epoch 887
test_train
len train dataset:  4480
train mean loss=161.60660552978516
test_test
len test dataset:  1120
test mean loss=151.26946640014648
epoch 888
test_train
len train dataset:  4480
train mean loss=162.48414783477784
test_test
len test dataset:  1120
test mean loss=149.93559951782225
epoch 889
test_train
len train dataset:  4480
train mean loss=165.8792995452881
test_test
len test dataset:  1120
test mean loss=151.4151924133301
epoch 890
test_train
len train dataset:  4480
train mean loss=170.3159730911

epoch 939
test_train
len train dataset:  4480
train mean loss=166.4258867263794
test_test
len test dataset:  1120
test mean loss=150.71775970458984
epoch 940
test_train
len train dataset:  4480
train mean loss=155.78936157226562
test_test
len test dataset:  1120
test mean loss=151.04424285888672
epoch 941
test_train
len train dataset:  4480
train mean loss=161.40353870391846
test_test
len test dataset:  1120
test mean loss=151.8754623413086
epoch 942
test_train
len train dataset:  4480
train mean loss=159.46789836883545
test_test
len test dataset:  1120
test mean loss=149.5735641479492
epoch 943
test_train
len train dataset:  4480
train mean loss=162.8777763366699
test_test
len test dataset:  1120
test mean loss=151.00192489624024
epoch 944
test_train
len train dataset:  4480
train mean loss=163.94127807617187
test_test
len test dataset:  1120
test mean loss=149.98704223632814
epoch 945
test_train
len train dataset:  4480
train mean loss=164.665181350708
test_test
len test dataset:  11

len train dataset:  4480
train mean loss=161.35908298492433
test_test
len test dataset:  1120
test mean loss=148.6080062866211
epoch 995
test_train
len train dataset:  4480
train mean loss=154.3390172958374
test_test
len test dataset:  1120
test mean loss=151.20831527709962
epoch 996
test_train
len train dataset:  4480
train mean loss=153.46005935668944
test_test
len test dataset:  1120
test mean loss=147.84037399291992
epoch 997
test_train
len train dataset:  4480
train mean loss=159.67309474945068
test_test
len test dataset:  1120
test mean loss=149.5574935913086
epoch 998
test_train
len train dataset:  4480
train mean loss=161.6956003189087
test_test
len test dataset:  1120
test mean loss=148.88565368652343
epoch 999
test_train
len train dataset:  4480
train mean loss=169.92748336791993
test_test
len test dataset:  1120
test mean loss=149.08398666381837
epoch 1000
test_train
len train dataset:  4480
train mean loss=156.81938133239746
test_test
len test dataset:  1120
test mean loss=

In [18]:
plt.figure(figsize=(6,6))      #グラフ描画用
ylim = max(max_train_loss_value, max_test_loss_value)

baseEPOCH = 100
num_epoch = 20 #100単位
act_num_epoch = baseEPOCH * num_epoch

#以下グラフ描画
plt.plot(range(act_num_epoch), train_loss_value)
plt.plot(range(act_num_epoch), test_loss_value, c='#00ff00')
plt.xlim(0, act_num_epoch)
plt.ylim(0, ylim)
#plt.ylim(0, 10)
plt.xlabel('EPOCH')
plt.ylabel('LOSS')
plt.legend(['train loss', 'test loss'])
plt.title('loss')
plt.savefig(PATH + "\\loss_image_" + str(num_epoch) + "_2.png")
plt.clf()

<Figure size 432x432 with 0 Axes>

In [39]:
stradrs = str(train_adrs[998][0])
print(type(train_adrs[0]))

<class 'torch.Tensor'>


In [16]:
import matplotlib
#matplotlib.use('Agg')
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
sns.set_style("darkgrid")

#結果画像出力
target_epoch = 999

HAND_PNT_NUM = 21

#fig = plt.figure(figsize=(6,6))
fig = plt.figure()
for i in range(len(train_input_value[target_epoch])):
    input_x = []
    input_y = []
    input_z = []
    mid = []
    output_x = []
    output_y = []
    output_z = []
    connect_x = []
    connect_y = []
    connect_z = []
    ax = fig.add_subplot(111, projection='3d')
    for j in range(int(len(train_input_value[target_epoch][0])/4)):
        input_x.append(train_input_value[target_epoch][i][j*4+0].item())
        input_y.append(-1*train_input_value[target_epoch][i][j*4+1].item())
        input_z.append(train_input_value[target_epoch][i][j*4+2].item())
        output_x.append(train_output_value[target_epoch][i][j*3+0].item())
        output_y.append(-1*train_output_value[target_epoch][i][j*3+1].item())
        output_z.append(train_output_value[target_epoch][i][j*3+2].item())
    x_max = max([max(input_x), max(output_x)], default = -10000)
    x_min = min([min(input_x), min(output_x)], default = 10000)
    y_max = max([max(input_y), max(output_y)], default = -10000)
    y_min = min([min(input_y), min(output_y)], default = 10000)
    z_max = max([max(input_z), max(output_z)], default = -10000)
    z_min = min([min(input_z), min(output_z)], default = 10000)
    
    #print(x_min, x_max, y_min, y_max, z_min, z_max, "(fixed: x_min, x_max, y_min, y_max, z_min, z_max)")
    
    #点が描画範囲内かどうか
    isInputPointsIn = [False] * HAND_PNT_NUM
    isOutputPointsIn = [False] * HAND_PNT_NUM
    #print(isPointsIn)
    
    for p in range(HAND_PNT_NUM):
        if x_min <= input_x[p] <= x_max and y_min <= input_y[p] <= y_max and z_min <= input_z[p] <= z_max:
            isInputPointsIn[p] = True
        if x_min <= output_x[p] <= x_max and y_min <= output_y[p] <= y_max and z_min <= output_z[p] <= z_max:
            isOutputPointsIn[p] = True
    
    #各点をプロット
    ax.scatter(input_x[0], input_y[0], zs=input_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(input_x[1], input_y[1], zs=input_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(input_x[2], input_y[2], zs=input_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(input_x[3], input_y[3], zs=input_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(input_x[4], input_y[4], zs=input_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(input_x[5], input_y[5], zs=input_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(input_x[6], input_y[6], zs=input_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(input_x[7], input_y[7], zs=input_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(input_x[8], input_y[8], zs=input_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(input_x[9], input_y[9], zs=input_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(input_x[10], input_y[10], zs=input_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(input_x[11], input_y[11], zs=input_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(input_x[12], input_y[12], zs=input_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(input_x[13], input_y[13], zs=input_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(input_x[14], input_y[14], zs=input_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(input_x[15], input_y[15], zs=input_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(input_x[16], input_y[16], zs=input_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(input_x[17], input_y[17], zs=input_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(input_x[18], input_y[18], zs=input_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(input_x[19], input_y[19], zs=input_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(input_x[20], input_y[20], zs=input_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isInputPointsIn[0] and isInputPointsIn[1]:
        ax.plot([input_x[0],input_x[1]], [input_y[0],input_y[1]], [input_z[0],input_z[1]], zdir='y', c='#cc0000')
    if isInputPointsIn[1] and isInputPointsIn[2]:
        ax.plot([input_x[1],input_x[2]], [input_y[1],input_y[2]], [input_z[1],input_z[2]], zdir='y', c='#b30000')
    if isInputPointsIn[2] and isInputPointsIn[3]:
        ax.plot([input_x[2],input_x[3]], [input_y[2],input_y[3]], [input_z[2],input_z[3]], zdir='y', c='#e60000')
    if isInputPointsIn[3] and isInputPointsIn[4]:
        ax.plot([input_x[3],input_x[4]], [input_y[3],input_y[4]], [input_z[3],input_z[4]], zdir='y', c='#ff0000')
    if isInputPointsIn[0] and isInputPointsIn[5]:
        ax.plot([input_x[0],input_x[5]], [input_y[0],input_y[5]], [input_z[0],input_z[5]], zdir='y', c='#a3cc00')
    if isInputPointsIn[5] and isInputPointsIn[6]:
        ax.plot([input_x[5],input_x[6]], [input_y[5],input_y[6]], [input_z[5],input_z[6]], zdir='y', c='#8fb300')
    if isInputPointsIn[6] and isInputPointsIn[7]:        
        ax.plot([input_x[6],input_x[7]], [input_y[6],input_y[7]], [input_z[6],input_z[7]], zdir='y', c='#b8e600')
    if isInputPointsIn[7] and isInputPointsIn[8]:
        ax.plot([input_x[7],input_x[8]], [input_y[7],input_y[8]], [input_z[7],input_z[8]], zdir='y', c='#ccff00')
    if isInputPointsIn[0] and isInputPointsIn[9]:
        ax.plot([input_x[0],input_x[9]], [input_y[0],input_y[9]], [input_z[0],input_z[9]], zdir='y', c='#00cc52')
    if isInputPointsIn[9] and isInputPointsIn[10]:
        ax.plot([input_x[9],input_x[10]], [input_y[9],input_y[10]], [input_z[9],input_z[10]], zdir='y', c='#00b347')
    if isInputPointsIn[10] and isInputPointsIn[11]:
        ax.plot([input_x[10],input_x[11]], [input_y[10],input_y[11]], [input_z[10],input_z[11]], zdir='y', c='#00e65c')
    if isInputPointsIn[11] and isInputPointsIn[12]:
        ax.plot([input_x[11],input_x[12]], [input_y[11],input_y[12]], [input_z[11],input_z[12]], zdir='y', c='#00ff66')
    if isInputPointsIn[0] and isInputPointsIn[13]:
        ax.plot([input_x[0],input_x[13]], [input_y[0],input_y[13]], [input_z[0],input_z[13]], zdir='y', c='#0052cc')
    if isInputPointsIn[13] and isInputPointsIn[14]:
        ax.plot([input_x[13],input_x[14]], [input_y[13],input_y[14]], [input_z[13],input_z[14]], zdir='y', c='#0047b3')
    if isInputPointsIn[14] and isInputPointsIn[15]:
        ax.plot([input_x[14],input_x[15]], [input_y[14],input_y[15]], [input_z[14],input_z[15]], zdir='y', c='#005ce6')
    if isInputPointsIn[15] and isInputPointsIn[16]:
        ax.plot([input_x[15],input_x[16]], [input_y[15],input_y[16]], [input_z[15],input_z[16]], zdir='y', c='#0066ff')
    if isInputPointsIn[0] and isInputPointsIn[17]:
        ax.plot([input_x[0],input_x[17]], [input_y[0],input_y[17]], [input_z[0],input_z[17]], zdir='y', c='#a300cc')
    if isInputPointsIn[17] and isInputPointsIn[18]:
        ax.plot([input_x[17],input_x[18]], [input_y[17],input_y[18]], [input_z[17],input_z[18]], zdir='y', c='#8f00b3')
    if isInputPointsIn[18] and isInputPointsIn[19]:
        ax.plot([input_x[18],input_x[19]], [input_y[18],input_y[19]], [input_z[18],input_z[19]], zdir='y', c='#b800e6')
    if isInputPointsIn[19] and isInputPointsIn[20]:
        ax.plot([input_x[19],input_x[20]], [input_y[19],input_y[20]], [input_z[19],input_z[20]], zdir='y', c='#cc00ff')
    
    stradrs = str(train_adrs[target_epoch][i].item()) #202011221813290002
    #print(stradrs[:8])
    #print(stradrs[8:14])
    #print(stradrs[14:])
    d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ax.set_title('input(train data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\Input_train_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300, transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
    
    ax = fig.add_subplot(111, projection='3d')
    #各点をプロット
    ax.scatter(output_x[0], output_y[0], zs=output_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(output_x[1], output_y[1], zs=output_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(output_x[2], output_y[2], zs=output_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(output_x[3], output_y[3], zs=output_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(output_x[4], output_y[4], zs=output_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(output_x[5], output_y[5], zs=output_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(output_x[6], output_y[6], zs=output_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(output_x[7], output_y[7], zs=output_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(output_x[8], output_y[8], zs=output_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(output_x[9], output_y[9], zs=output_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(output_x[10], output_y[10], zs=output_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(output_x[11], output_y[11], zs=output_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(output_x[12], output_y[12], zs=output_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(output_x[13], output_y[13], zs=output_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(output_x[14], output_y[14], zs=output_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(output_x[15], output_y[15], zs=output_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(output_x[16], output_y[16], zs=output_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(output_x[17], output_y[17], zs=output_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(output_x[18], output_y[18], zs=output_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(output_x[19], output_y[19], zs=output_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(output_x[20], output_y[20], zs=output_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isOutputPointsIn[0] and isOutputPointsIn[1]:
        ax.plot([output_x[0],output_x[1]], [output_y[0],output_y[1]], [output_z[0],output_z[1]], zdir='y', c='#cc0000')
    if isOutputPointsIn[1] and isOutputPointsIn[2]:
        ax.plot([output_x[1],output_x[2]], [output_y[1],output_y[2]], [output_z[1],output_z[2]], zdir='y', c='#b30000')
    if isOutputPointsIn[2] and isOutputPointsIn[3]:
        ax.plot([output_x[2],output_x[3]], [output_y[2],output_y[3]], [output_z[2],output_z[3]], zdir='y', c='#e60000')
    if isOutputPointsIn[3] and isOutputPointsIn[4]:
        ax.plot([output_x[3],output_x[4]], [output_y[3],output_y[4]], [output_z[3],output_z[4]], zdir='y', c='#ff0000')
    if isOutputPointsIn[0] and isOutputPointsIn[5]:
        ax.plot([output_x[0],output_x[5]], [output_y[0],output_y[5]], [output_z[0],output_z[5]], zdir='y', c='#a3cc00')
    if isOutputPointsIn[5] and isOutputPointsIn[6]:
        ax.plot([output_x[5],output_x[6]], [output_y[5],output_y[6]], [output_z[5],output_z[6]], zdir='y', c='#8fb300')
    if isOutputPointsIn[6] and isOutputPointsIn[7]:        
        ax.plot([output_x[6],output_x[7]], [output_y[6],output_y[7]], [output_z[6],output_z[7]], zdir='y', c='#b8e600')
    if isOutputPointsIn[7] and isOutputPointsIn[8]:
        ax.plot([output_x[7],output_x[8]], [output_y[7],output_y[8]], [output_z[7],output_z[8]], zdir='y', c='#ccff00')
    if isOutputPointsIn[0] and isOutputPointsIn[9]:
        ax.plot([output_x[0],output_x[9]], [output_y[0],output_y[9]], [output_z[0],output_z[9]], zdir='y', c='#00cc52')
    if isOutputPointsIn[9] and isOutputPointsIn[10]:
        ax.plot([output_x[9],output_x[10]], [output_y[9],output_y[10]], [output_z[9],output_z[10]], zdir='y', c='#00b347')
    if isOutputPointsIn[10] and isOutputPointsIn[11]:
        ax.plot([output_x[10],output_x[11]], [output_y[10],output_y[11]], [output_z[10],output_z[11]], zdir='y', c='#00e65c')
    if isOutputPointsIn[11] and isOutputPointsIn[12]:
        ax.plot([output_x[11],output_x[12]], [output_y[11],output_y[12]], [output_z[11],output_z[12]], zdir='y', c='#00ff66')
    if isOutputPointsIn[0] and isOutputPointsIn[13]:
        ax.plot([output_x[0],output_x[13]], [output_y[0],output_y[13]], [output_z[0],output_z[13]], zdir='y', c='#0052cc')
    if isOutputPointsIn[13] and isOutputPointsIn[14]:
        ax.plot([output_x[13],output_x[14]], [output_y[13],output_y[14]], [output_z[13],output_z[14]], zdir='y', c='#0047b3')
    if isOutputPointsIn[14] and isOutputPointsIn[15]:
        ax.plot([output_x[14],output_x[15]], [output_y[14],output_y[15]], [output_z[14],output_z[15]], zdir='y', c='#005ce6')
    if isOutputPointsIn[15] and isOutputPointsIn[16]:
        ax.plot([output_x[15],output_x[16]], [output_y[15],output_y[16]], [output_z[15],output_z[16]], zdir='y', c='#0066ff')
    if isOutputPointsIn[0] and isOutputPointsIn[17]:
        ax.plot([output_x[0],output_x[17]], [output_y[0],output_y[17]], [output_z[0],output_z[17]], zdir='y', c='#a300cc')
    if isOutputPointsIn[17] and isOutputPointsIn[18]:
        ax.plot([output_x[17],output_x[18]], [output_y[17],output_y[18]], [output_z[17],output_z[18]], zdir='y', c='#8f00b3')
    if isOutputPointsIn[18] and isOutputPointsIn[19]:
        ax.plot([output_x[18],output_x[19]], [output_y[18],output_y[19]], [output_z[18],output_z[19]], zdir='y', c='#b800e6')
    if isOutputPointsIn[19] and isOutputPointsIn[20]:
        ax.plot([output_x[19],output_x[20]], [output_y[19],output_y[20]], [output_z[19],output_z[20]], zdir='y', c='#cc00ff')
    #ax.scatter(input_x, input_y, c='red', label = 'input')
    #ax.scatter(output_x, output_y, c='blue', label = 'output')
    
    #for l in range(len(input_x)):
    #    connect_x = [input_x[l], output_x[l]]
    #    connect_y = [input_y[l], output_y[l]]
    #    connect_z = [input_z[l], output_z[l]]
    #    ax.plot(connect_x,connect_y,connect_z,zdir='y',color='gray')
    stradrs = str(train_adrs[target_epoch][i].item()) #202011221813290002
    #print(stradrs[:8])
    #print(stradrs[8:14])
    #print(stradrs[14:])
    d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ax.set_title('output(train data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\Output_train_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300,transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
for i in range(len(test_input_value[target_epoch])):
    input_x = []
    input_y = []
    input_z = []
    mid = []
    output_x = []
    output_y = []
    output_z = []
    connect_x = []
    connect_y = []
    connect_z = []
    ax = fig.add_subplot(111, projection='3d')
    for j in range(int(len(test_input_value[target_epoch][0])/4)):
        input_x.append(test_input_value[target_epoch][i][j*4+0].item())
        input_y.append(-1*test_input_value[target_epoch][i][j*4+1].item())
        input_z.append(test_input_value[target_epoch][i][j*4+2].item())
        output_x.append(test_output_value[target_epoch][i][j*3+0].item())
        output_y.append(-1*test_output_value[target_epoch][i][j*3+1].item())
        output_z.append(test_output_value[target_epoch][i][j*3+2].item())
    x_max = max([max(input_x), max(output_x)], default = -10000)
    x_min = min([min(input_x), min(output_x)], default = 10000)
    y_max = max([max(input_y), max(output_y)], default = -10000)
    y_min = min([min(input_y), min(output_y)], default = 10000)
    z_max = max([max(input_z), max(output_z)], default = -10000)
    z_min = min([min(input_z), min(output_z)], default = 10000)
    
    #print(x_min, x_max, y_min, y_max, z_min, z_max, "(fixed: x_min, x_max, y_min, y_max, z_min, z_max)")
    
    #点が描画範囲内かどうか
    isInputPointsIn = [False] * HAND_PNT_NUM
    isOutputPointsIn = [False] * HAND_PNT_NUM
    #print(isPointsIn)
    
    for p in range(HAND_PNT_NUM):
        if x_min <= input_x[p] <= x_max and y_min <= input_y[p] <= y_max and z_min <= input_z[p] <= z_max:
            isInputPointsIn[p] = True
        if x_min <= output_x[p] <= x_max and y_min <= output_y[p] <= y_max and z_min <= output_z[p] <= z_max:
            isOutputPointsIn[p] = True
    
    #各点をプロット
    ax.scatter(input_x[0], input_y[0], zs=input_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(input_x[1], input_y[1], zs=input_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(input_x[2], input_y[2], zs=input_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(input_x[3], input_y[3], zs=input_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(input_x[4], input_y[4], zs=input_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(input_x[5], input_y[5], zs=input_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(input_x[6], input_y[6], zs=input_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(input_x[7], input_y[7], zs=input_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(input_x[8], input_y[8], zs=input_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(input_x[9], input_y[9], zs=input_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(input_x[10], input_y[10], zs=input_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(input_x[11], input_y[11], zs=input_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(input_x[12], input_y[12], zs=input_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(input_x[13], input_y[13], zs=input_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(input_x[14], input_y[14], zs=input_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(input_x[15], input_y[15], zs=input_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(input_x[16], input_y[16], zs=input_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(input_x[17], input_y[17], zs=input_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(input_x[18], input_y[18], zs=input_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(input_x[19], input_y[19], zs=input_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(input_x[20], input_y[20], zs=input_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isInputPointsIn[0] and isInputPointsIn[1]:
        ax.plot([input_x[0],input_x[1]], [input_y[0],input_y[1]], [input_z[0],input_z[1]], zdir='y', c='#cc0000')
    if isInputPointsIn[1] and isInputPointsIn[2]:
        ax.plot([input_x[1],input_x[2]], [input_y[1],input_y[2]], [input_z[1],input_z[2]], zdir='y', c='#b30000')
    if isInputPointsIn[2] and isInputPointsIn[3]:
        ax.plot([input_x[2],input_x[3]], [input_y[2],input_y[3]], [input_z[2],input_z[3]], zdir='y', c='#e60000')
    if isInputPointsIn[3] and isInputPointsIn[4]:
        ax.plot([input_x[3],input_x[4]], [input_y[3],input_y[4]], [input_z[3],input_z[4]], zdir='y', c='#ff0000')
    if isInputPointsIn[0] and isInputPointsIn[5]:
        ax.plot([input_x[0],input_x[5]], [input_y[0],input_y[5]], [input_z[0],input_z[5]], zdir='y', c='#a3cc00')
    if isInputPointsIn[5] and isInputPointsIn[6]:
        ax.plot([input_x[5],input_x[6]], [input_y[5],input_y[6]], [input_z[5],input_z[6]], zdir='y', c='#8fb300')
    if isInputPointsIn[6] and isInputPointsIn[7]:        
        ax.plot([input_x[6],input_x[7]], [input_y[6],input_y[7]], [input_z[6],input_z[7]], zdir='y', c='#b8e600')
    if isInputPointsIn[7] and isInputPointsIn[8]:
        ax.plot([input_x[7],input_x[8]], [input_y[7],input_y[8]], [input_z[7],input_z[8]], zdir='y', c='#ccff00')
    if isInputPointsIn[0] and isInputPointsIn[9]:
        ax.plot([input_x[0],input_x[9]], [input_y[0],input_y[9]], [input_z[0],input_z[9]], zdir='y', c='#00cc52')
    if isInputPointsIn[9] and isInputPointsIn[10]:
        ax.plot([input_x[9],input_x[10]], [input_y[9],input_y[10]], [input_z[9],input_z[10]], zdir='y', c='#00b347')
    if isInputPointsIn[10] and isInputPointsIn[11]:
        ax.plot([input_x[10],input_x[11]], [input_y[10],input_y[11]], [input_z[10],input_z[11]], zdir='y', c='#00e65c')
    if isInputPointsIn[11] and isInputPointsIn[12]:
        ax.plot([input_x[11],input_x[12]], [input_y[11],input_y[12]], [input_z[11],input_z[12]], zdir='y', c='#00ff66')
    if isInputPointsIn[0] and isInputPointsIn[13]:
        ax.plot([input_x[0],input_x[13]], [input_y[0],input_y[13]], [input_z[0],input_z[13]], zdir='y', c='#0052cc')
    if isInputPointsIn[13] and isInputPointsIn[14]:
        ax.plot([input_x[13],input_x[14]], [input_y[13],input_y[14]], [input_z[13],input_z[14]], zdir='y', c='#0047b3')
    if isInputPointsIn[14] and isInputPointsIn[15]:
        ax.plot([input_x[14],input_x[15]], [input_y[14],input_y[15]], [input_z[14],input_z[15]], zdir='y', c='#005ce6')
    if isInputPointsIn[15] and isInputPointsIn[16]:
        ax.plot([input_x[15],input_x[16]], [input_y[15],input_y[16]], [input_z[15],input_z[16]], zdir='y', c='#0066ff')
    if isInputPointsIn[0] and isInputPointsIn[17]:
        ax.plot([input_x[0],input_x[17]], [input_y[0],input_y[17]], [input_z[0],input_z[17]], zdir='y', c='#a300cc')
    if isInputPointsIn[17] and isInputPointsIn[18]:
        ax.plot([input_x[17],input_x[18]], [input_y[17],input_y[18]], [input_z[17],input_z[18]], zdir='y', c='#8f00b3')
    if isInputPointsIn[18] and isInputPointsIn[19]:
        ax.plot([input_x[18],input_x[19]], [input_y[18],input_y[19]], [input_z[18],input_z[19]], zdir='y', c='#b800e6')
    if isInputPointsIn[19] and isInputPointsIn[20]:
        ax.plot([input_x[19],input_x[20]], [input_y[19],input_y[20]], [input_z[19],input_z[20]], zdir='y', c='#cc00ff')
    
    stradrs = str(test_adrs[target_epoch][i].item()) #202011221813290002
    #print(stradrs[:8])
    #print(stradrs[8:14])
    #print(stradrs[14:])
    d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ax.set_title('input(test data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\Input_test_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300, transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
    
    ax = fig.add_subplot(111, projection='3d')
    #各点をプロット
    ax.scatter(output_x[0], output_y[0], zs=output_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(output_x[1], output_y[1], zs=output_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(output_x[2], output_y[2], zs=output_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(output_x[3], output_y[3], zs=output_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(output_x[4], output_y[4], zs=output_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(output_x[5], output_y[5], zs=output_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(output_x[6], output_y[6], zs=output_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(output_x[7], output_y[7], zs=output_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(output_x[8], output_y[8], zs=output_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(output_x[9], output_y[9], zs=output_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(output_x[10], output_y[10], zs=output_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(output_x[11], output_y[11], zs=output_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(output_x[12], output_y[12], zs=output_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(output_x[13], output_y[13], zs=output_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(output_x[14], output_y[14], zs=output_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(output_x[15], output_y[15], zs=output_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(output_x[16], output_y[16], zs=output_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(output_x[17], output_y[17], zs=output_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(output_x[18], output_y[18], zs=output_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(output_x[19], output_y[19], zs=output_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(output_x[20], output_y[20], zs=output_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isOutputPointsIn[0] and isOutputPointsIn[1]:
        ax.plot([output_x[0],output_x[1]], [output_y[0],output_y[1]], [output_z[0],output_z[1]], zdir='y', c='#cc0000')
    if isOutputPointsIn[1] and isOutputPointsIn[2]:
        ax.plot([output_x[1],output_x[2]], [output_y[1],output_y[2]], [output_z[1],output_z[2]], zdir='y', c='#b30000')
    if isOutputPointsIn[2] and isOutputPointsIn[3]:
        ax.plot([output_x[2],output_x[3]], [output_y[2],output_y[3]], [output_z[2],output_z[3]], zdir='y', c='#e60000')
    if isOutputPointsIn[3] and isOutputPointsIn[4]:
        ax.plot([output_x[3],output_x[4]], [output_y[3],output_y[4]], [output_z[3],output_z[4]], zdir='y', c='#ff0000')
    if isOutputPointsIn[0] and isOutputPointsIn[5]:
        ax.plot([output_x[0],output_x[5]], [output_y[0],output_y[5]], [output_z[0],output_z[5]], zdir='y', c='#a3cc00')
    if isOutputPointsIn[5] and isOutputPointsIn[6]:
        ax.plot([output_x[5],output_x[6]], [output_y[5],output_y[6]], [output_z[5],output_z[6]], zdir='y', c='#8fb300')
    if isOutputPointsIn[6] and isOutputPointsIn[7]:        
        ax.plot([output_x[6],output_x[7]], [output_y[6],output_y[7]], [output_z[6],output_z[7]], zdir='y', c='#b8e600')
    if isOutputPointsIn[7] and isOutputPointsIn[8]:
        ax.plot([output_x[7],output_x[8]], [output_y[7],output_y[8]], [output_z[7],output_z[8]], zdir='y', c='#ccff00')
    if isOutputPointsIn[0] and isOutputPointsIn[9]:
        ax.plot([output_x[0],output_x[9]], [output_y[0],output_y[9]], [output_z[0],output_z[9]], zdir='y', c='#00cc52')
    if isOutputPointsIn[9] and isOutputPointsIn[10]:
        ax.plot([output_x[9],output_x[10]], [output_y[9],output_y[10]], [output_z[9],output_z[10]], zdir='y', c='#00b347')
    if isOutputPointsIn[10] and isOutputPointsIn[11]:
        ax.plot([output_x[10],output_x[11]], [output_y[10],output_y[11]], [output_z[10],output_z[11]], zdir='y', c='#00e65c')
    if isOutputPointsIn[11] and isOutputPointsIn[12]:
        ax.plot([output_x[11],output_x[12]], [output_y[11],output_y[12]], [output_z[11],output_z[12]], zdir='y', c='#00ff66')
    if isOutputPointsIn[0] and isOutputPointsIn[13]:
        ax.plot([output_x[0],output_x[13]], [output_y[0],output_y[13]], [output_z[0],output_z[13]], zdir='y', c='#0052cc')
    if isOutputPointsIn[13] and isOutputPointsIn[14]:
        ax.plot([output_x[13],output_x[14]], [output_y[13],output_y[14]], [output_z[13],output_z[14]], zdir='y', c='#0047b3')
    if isOutputPointsIn[14] and isOutputPointsIn[15]:
        ax.plot([output_x[14],output_x[15]], [output_y[14],output_y[15]], [output_z[14],output_z[15]], zdir='y', c='#005ce6')
    if isOutputPointsIn[15] and isOutputPointsIn[16]:
        ax.plot([output_x[15],output_x[16]], [output_y[15],output_y[16]], [output_z[15],output_z[16]], zdir='y', c='#0066ff')
    if isOutputPointsIn[0] and isOutputPointsIn[17]:
        ax.plot([output_x[0],output_x[17]], [output_y[0],output_y[17]], [output_z[0],output_z[17]], zdir='y', c='#a300cc')
    if isOutputPointsIn[17] and isOutputPointsIn[18]:
        ax.plot([output_x[17],output_x[18]], [output_y[17],output_y[18]], [output_z[17],output_z[18]], zdir='y', c='#8f00b3')
    if isOutputPointsIn[18] and isOutputPointsIn[19]:
        ax.plot([output_x[18],output_x[19]], [output_y[18],output_y[19]], [output_z[18],output_z[19]], zdir='y', c='#b800e6')
    if isOutputPointsIn[19] and isOutputPointsIn[20]:
        ax.plot([output_x[19],output_x[20]], [output_y[19],output_y[20]], [output_z[19],output_z[20]], zdir='y', c='#cc00ff')
    #ax.scatter(input_x, input_y, c='red', label = 'input')
    #ax.scatter(output_x, output_y, c='blue', label = 'output')
    
    #for l in range(len(input_x)):
    #    connect_x = [input_x[l], output_x[l]]
    #    connect_y = [input_y[l], output_y[l]]
    #    connect_z = [input_z[l], output_z[l]]
    #    ax.plot(connect_x,connect_y,connect_z,zdir='y',color='gray')
    stradrs = str(test_adrs[target_epoch][i].item()) #11221813290002
    #print(stradrs[:8])
    #print(stradrs[8:14])
    #print(stradrs[14:])
    d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ax.set_title('output(test data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\Output_test_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300, transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
plt.close()

In [27]:
#出力値保存

with open(PATH + "\\network_outs.csv", mode='w') as f:
    writer = csv.writer(f)
    writer.writerow(["train_test"," "])
    for i in range(len(train_input_value)):                      #epoch数 loop 1000
        writer.writerow(["epoch:" + str(i+1)," "])
        for j in range(len(train_input_value[i])):               #batch size数 loop 10
            writer.writerow(["batch:" + str(j+1)," "])
            input_x = []
            input_y = []
            input_z = []
            output_x = []
            output_y = []
            output_z = []
            middle = []
            for k in range(int(len(train_input_value[i][j])/4)): #data size / 2 loop (x, y分離) 21(42)
                input_x.append(train_input_value[i][j][k*4+0].item())
                input_y.append(train_input_value[i][j][k*4+1].item())
                input_z.append(train_input_value[i][j][k*4+2].item())
                output_x.append(train_output_value[i][j][k*3+0].item())
                output_y.append(train_output_value[i][j][k*3+1].item())
                output_z.append(train_output_value[i][j][k*3+2].item())
            for l in range(len(train_mid_out_value[i][j])):
                middle.append(train_mid_out_value[i][j][l].item())
            writer.writerow(input_x)
            writer.writerow(input_y)
            writer.writerow(output_x)
            writer.writerow(output_y)
            writer.writerow(middle)
    writer.writerow(["test_test"," "])
    for i in range(len(test_input_value)):                      #epoch数 loop 1000
        writer.writerow(["epoch:" + str(i+1)," "])
        for j in range(len(test_input_value[i])):               #batch size数 loop 10
            writer.writerow(["batch:" + str(j+1)," "])
            input_x = []
            input_y = []
            input_z = []
            output_x = []
            output_y = []
            output_z = []
            middle = []
            for k in range(int(len(test_input_value[i][j])/4)): #data size / 2 loop (x, y分離) 21(42)
                input_x.append(test_input_value[i][j][k*4+0].item())
                input_y.append(test_input_value[i][j][k*4+1].item())
                input_z.append(test_input_value[i][j][k*4+2].item())
                output_x.append(test_output_value[i][j][k*3+0].item())
                output_y.append(test_output_value[i][j][k*3+1].item())
                output_z.append(test_output_value[i][j][k*3+2].item())
            for l in range(len(test_mid_out_value[i][j])):
                middle.append(test_mid_out_value[i][j][l].item())
            writer.writerow(input_x)
            writer.writerow(input_y)
            writer.writerow(output_x)
            writer.writerow(output_y)
            writer.writerow(middle)
    print("fin save.")

fin save.


In [29]:
torch.save(net.state_dict(), PATH + "\\hand_AE_model")
print(net.state_dict()["dense_enc1.weight"])
for p in net.parameters():
    print(p)

tensor([[ 5.3373e-02, -5.8840e-02,  1.1394e-02,  ..., -8.6241e-02,
          8.0329e-02,  2.4244e-03],
        [-5.6551e-02, -5.4824e-02,  5.3382e-02,  ..., -6.9136e-02,
         -6.8530e-02,  1.2134e-02],
        [-4.4319e-02, -6.3197e-05, -1.8928e-02,  ...,  9.0667e-03,
          2.1744e-02, -6.0273e-02],
        ...,
        [-5.6997e-03,  5.5421e-02, -4.7828e-02,  ..., -4.1679e-02,
         -5.9505e-02, -1.3725e-02],
        [ 5.7143e-02,  1.0077e-02, -5.9345e-04,  ...,  5.6414e-02,
         -3.8345e-03,  5.7085e-02],
        [ 2.0989e-02, -2.9862e-02, -3.2242e-02,  ...,  1.5728e-03,
          2.6473e-05, -6.3841e-02]], device='cuda:0')
Parameter containing:
tensor([[[[-0.1664, -0.1607, -0.0110],
          [ 0.1621,  0.0476, -0.0337],
          [ 0.3198,  0.0540,  0.1888]]],


        [[[ 0.2591, -0.3306,  0.1396],
          [-0.2651,  0.2726, -0.2523],
          [ 0.0979, -0.1826,  0.2165]]],


        [[[-0.1440,  0.1066,  0.1338],
          [-0.2165,  0.2444,  0.2687],
         

       requires_grad=True)
Parameter containing:
tensor([-0.0647, -0.0468, -0.0309, -0.0596,  0.0624, -0.0054,  0.0242,  0.0231,
        -0.0635,  0.0486, -0.0592, -0.0234, -0.0329, -0.0788, -0.0119,  0.0539,
        -0.0275,  0.0242,  0.0075,  0.0219,  0.0689, -0.0256,  0.0761, -0.0797,
         0.0317, -0.0323,  0.0100,  0.0131,  0.0728, -0.0153,  0.0575,  0.0163],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.0664, 0.1608, 0.1247], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.

       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.0295, -0.0398,  0.0245,  ..., -0.0381,  0.0297, -0.0410],
        [-0.0282, -0.0248, -0.0241,  ...,  0.0512, -0.0429,  0.0446],
        [-0.0346, -0.0184,  0.0115,  ...,  0.0481, -0.0555,  0.0216],
        ...,
        [-0.0417,  0.0152,  0.0095,  ...,  0.0210,  0.0120, -0.0227],
        [-0.0440, -0.0304, -0.0053,  ..., -0.0078,  0.0504,  0.0368],
        [-0.0368,  0.0262,  0.0259,  ..., -0.0351,  0.0351,  0.0320]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.0419,  0.0317,  0.0521, -0.0243, -0.0343, -0.0205, -0.0347, -0.0422,
        -0.0454,  0.0010, -0.0244, -0.0575,  0.0550,  0.0394,  0.0125,  0.0334,
        -0.0227, -0.0457,  0.0177, -0.0193,  0.0437,  0.0142, -0.0280,  0.0136,
        -0.0196,  0.0216, -0.0371, -0.0416,  0.0243, -0.0405,  0.0429, -0.0281,
        -0.0141, -0.0573, -0.0280, -0.0491, -0.0056,  0.0153,  0.0392, -0.0551,
         0.0263,  0.0513,  0.0116,

       requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([[ 0.5187, -0.2988,  1.4476,  ...,  0.1303,  0.0928, -0.9674],
        [-0.0832,  1.2001,  0.1535,  ...,  0.6594, -0.7502, -0.7395],
        [ 0.9375,  1.4735,  1.2090,  ...,  0.7215,  0.0419, -0.2470],
        ...,
        [-0.0476, -0.2844,  0.1616,  ..., -0.4570, -1.2957,  1.15

       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0436,  0.0833,  0.2322, -0.0133,  0.1462,  0.0043,  0.2534,  0.0963,
         0.0353,  0.0103,  0.0434, -0.0412,  0.1905, -0.0035, -0.0105,  0.1098,
         0.3375,  0.0691], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.6082, 0.6897, 0.6957, 0.6479, 0.6981, 0.6703, 0.6909, 0.6514, 0.6851,
        0.6202, 0.6558, 0.6574, 0.6633, 0.6747, 0.6973, 0.6544, 0.6468, 0.6679],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0198,  0.0097, -0.0030,  0.0555,  0.0251,  0.0195,  0.0577,  0.0374,
         0.0024, -0.0044,  0.0091,  0.0195, -0.0204, -0.0021,  0.0028, -0.0211,
        -0.0238, -0.0182], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[-0.1558,  0.0337,  0.1086, -0.0715, -0.1420, -0.1795,  0.0124,  0.1123,
         -0.0378,  0.1228, -0.2082,  0.1233, -0.0392,  0.2142,  0.0556,  0.1519,
          0.0053, -0.0434],
        [-0.0019, -0.1944

          2.0398e-01]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.1252,  0.1744,  0.2952,  0.0562, -0.0786,  0.1625, -0.0018, -0.0542,
        -0.1067,  0.1325,  0.0263,  0.2706,  0.0822,  0.0700,  0.1375,  0.0189,
         0.1954, -0.0010, -0.0186,  0.1084,  0.0152,  0.0651, -0.0830,  0.1608,
        -0.1563,  0.3326,  0.2044, -0.0370, -0.0531,  0.3900, -0.0148,  0.0222],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.6849, 0.7011, 1.0705, 0.9071, 0.7563, 0.8175, 0.6797, 0.6734, 0.5955,
        0.8296, 1.1760, 0.9320, 1.0982, 1.0469, 0.9246, 0.7601, 1.0945, 0.7785,
        0.8391, 0.8002, 0.9288, 0.7015, 0.6537, 0.6354, 0.7138, 0.9870, 0.9218,
        0.6326, 0.6966, 1.0490, 0.8955, 0.9242], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([ 0.0965, -0.0068, -0.0769,  0.1653, -0.0371, -0.0885, -0.0966, -0.1051,
        -0.0113, -0.0445, -0.1192, -0.0749,  0.1258, -0.1510,  0.0622, -0.0712,
         0.2413

In [34]:
test = [-21,-20,24,16,-8,28,13,-26,-27,12,14,26,-15,-25,3,29,29,-6,-23,-19,4,29,-29,5,13,-26,6,-29,-27,12,15,25,-3,-29,-26,13,-27,-11,1,29,-20,-21]
testx2 = []
testx2.append(test)
testx2.append(test)

testm = [1.197573185,1.006648302,-1.906694531,0.34096396,0.966486633,0.236363918,0.758477569,-1.593988061]
testmx2 = []
testmx2.append(testm)
testmx2.append(testm)

testm = np.array(testmx2).astype('float32')#/100
#test_m = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
#test_mx2 = []
#test_mx2.append(test_m)
#test_mx2.append(test_m)
#test_m = np.array(test_mx2).astype('float32')
x = torch.from_numpy(testm).clone().to(device)
#m = torch.from_numpy(test_m).clone().to(device)
print("x: ", x[0], x.size())
#print("m: ", m[0], m.size())

#out, mid = net(x, m)
out, mid = net(z=x)

print("mid: ", mid[0], mid.size())
print("out: ", out[0], out.size())

x:  tensor([ 1.1976,  1.0066, -1.9067,  0.3410,  0.9665,  0.2364,  0.7585, -1.5940],
       device='cuda:0') torch.Size([2, 8])
decoder only
mid:  tensor([ 1.1976,  1.0066, -1.9067,  0.3410,  0.9665,  0.2364,  0.7585, -1.5940],
       device='cuda:0') torch.Size([2, 8])
out:  tensor([-2.6983e-04,  2.0072e-04, -3.0663e-04, -1.6508e-02,  8.2221e-03,
         2.3051e-02, -4.9086e-02,  1.9345e-02,  7.8549e-02, -9.2282e-02,
         4.5607e-02,  1.0071e-01, -1.0961e-01,  7.9863e-02,  1.4121e-01,
        -1.4789e-01, -6.0938e-02,  8.9856e-02, -2.1927e-01, -1.8011e-02,
         1.5419e-01, -2.5972e-01,  9.7074e-03,  1.6122e-01, -2.8264e-01,
         3.5855e-02,  1.7898e-01, -1.7168e-01, -4.9995e-02,  7.5108e-02,
        -2.5309e-01, -4.0018e-03,  1.2467e-01, -2.9490e-01,  3.1515e-02,
         1.2824e-01, -3.2153e-01,  5.5724e-02,  1.5552e-01, -1.7900e-01,
        -1.4698e-02,  8.0515e-02, -2.5479e-01,  2.7707e-02,  1.2940e-01,
        -2.9346e-01,  6.3404e-02,  1.3163e-01, -3.1732e-01,  9.090

In [40]:
import matplotlib
#matplotlib.use('Agg')
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
sns.set_style("darkgrid")

HAND_PNT_NUM = 21
fig2 = plt.figure()

output_x = []
output_y = []
output_z = []
connect_x = []
connect_y = []
connect_z = []
ax2 = fig2.add_subplot(111, projection='3d')
for j in range(int(len(out[0])/3)):
    output_x.append(out[0][j*3+0].item()*100)
    output_y.append(-1*out[0][j*3+1].item()*100)
    output_z.append(out[0][j*3+2].item()*100)
print(output_x)
print(output_y)
print(output_z)
x_max = max(output_x, default = -10000)
x_min = min(output_x, default = 10000)
y_max = max(output_y, default = -10000)
y_min = min(output_y, default = 10000)
z_max = max(output_z, default = -10000)
z_min = min(output_z, default = 10000)

print(x_min, x_max, y_min, y_max, z_min, z_max, "(fixed: x_min, x_max, y_min, y_max, z_min, z_max)")

#点が描画範囲内かどうか
isOutputPointsIn = [False] * HAND_PNT_NUM
#print(isPointsIn)

for p in range(HAND_PNT_NUM):
    if x_min <= output_x[p] <= x_max and y_min <= output_y[p] <= y_max and z_min <= output_z[p] <= z_max:
        isOutputPointsIn[p] = True
#各点をプロット
    ax2.scatter(output_x[0], output_y[0], zs=output_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax2.scatter(output_x[1], output_y[1], zs=output_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax2.scatter(output_x[2], output_y[2], zs=output_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax2.scatter(output_x[3], output_y[3], zs=output_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax2.scatter(output_x[4], output_y[4], zs=output_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax2.scatter(output_x[5], output_y[5], zs=output_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax2.scatter(output_x[6], output_y[6], zs=output_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax2.scatter(output_x[7], output_y[7], zs=output_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax2.scatter(output_x[8], output_y[8], zs=output_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax2.scatter(output_x[9], output_y[9], zs=output_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax2.scatter(output_x[10], output_y[10], zs=output_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax2.scatter(output_x[11], output_y[11], zs=output_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax2.scatter(output_x[12], output_y[12], zs=output_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax2.scatter(output_x[13], output_y[13], zs=output_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax2.scatter(output_x[14], output_y[14], zs=output_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax2.scatter(output_x[15], output_y[15], zs=output_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax2.scatter(output_x[16], output_y[16], zs=output_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax2.scatter(output_x[17], output_y[17], zs=output_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax2.scatter(output_x[18], output_y[18], zs=output_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax2.scatter(output_x[19], output_y[19], zs=output_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax2.scatter(output_x[20], output_y[20], zs=output_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isOutputPointsIn[0] and isOutputPointsIn[1]:
        ax2.plot([output_x[0],output_x[1]], [output_y[0],output_y[1]], [output_z[0],output_z[1]], zdir='y', c='#cc0000')
    if isOutputPointsIn[1] and isOutputPointsIn[2]:
        ax2.plot([output_x[1],output_x[2]], [output_y[1],output_y[2]], [output_z[1],output_z[2]], zdir='y', c='#b30000')
    if isOutputPointsIn[2] and isOutputPointsIn[3]:
        ax2.plot([output_x[2],output_x[3]], [output_y[2],output_y[3]], [output_z[2],output_z[3]], zdir='y', c='#e60000')
    if isOutputPointsIn[3] and isOutputPointsIn[4]:
        ax2.plot([output_x[3],output_x[4]], [output_y[3],output_y[4]], [output_z[3],output_z[4]], zdir='y', c='#ff0000')
    if isOutputPointsIn[0] and isOutputPointsIn[5]:
        ax2.plot([output_x[0],output_x[5]], [output_y[0],output_y[5]], [output_z[0],output_z[5]], zdir='y', c='#a3cc00')
    if isOutputPointsIn[5] and isOutputPointsIn[6]:
        ax2.plot([output_x[5],output_x[6]], [output_y[5],output_y[6]], [output_z[5],output_z[6]], zdir='y', c='#8fb300')
    if isOutputPointsIn[6] and isOutputPointsIn[7]:        
        ax2.plot([output_x[6],output_x[7]], [output_y[6],output_y[7]], [output_z[6],output_z[7]], zdir='y', c='#b8e600')
    if isOutputPointsIn[7] and isOutputPointsIn[8]:
        ax2.plot([output_x[7],output_x[8]], [output_y[7],output_y[8]], [output_z[7],output_z[8]], zdir='y', c='#ccff00')
    if isOutputPointsIn[0] and isOutputPointsIn[9]:
        ax2.plot([output_x[0],output_x[9]], [output_y[0],output_y[9]], [output_z[0],output_z[9]], zdir='y', c='#00cc52')
    if isOutputPointsIn[9] and isOutputPointsIn[10]:
        ax2.plot([output_x[9],output_x[10]], [output_y[9],output_y[10]], [output_z[9],output_z[10]], zdir='y', c='#00b347')
    if isOutputPointsIn[10] and isOutputPointsIn[11]:
        ax2.plot([output_x[10],output_x[11]], [output_y[10],output_y[11]], [output_z[10],output_z[11]], zdir='y', c='#00e65c')
    if isOutputPointsIn[11] and isOutputPointsIn[12]:
        ax2.plot([output_x[11],output_x[12]], [output_y[11],output_y[12]], [output_z[11],output_z[12]], zdir='y', c='#00ff66')
    if isOutputPointsIn[0] and isOutputPointsIn[13]:
        ax2.plot([output_x[0],output_x[13]], [output_y[0],output_y[13]], [output_z[0],output_z[13]], zdir='y', c='#0052cc')
    if isOutputPointsIn[13] and isOutputPointsIn[14]:
        ax2.plot([output_x[13],output_x[14]], [output_y[13],output_y[14]], [output_z[13],output_z[14]], zdir='y', c='#0047b3')
    if isOutputPointsIn[14] and isOutputPointsIn[15]:
        ax2.plot([output_x[14],output_x[15]], [output_y[14],output_y[15]], [output_z[14],output_z[15]], zdir='y', c='#005ce6')
    if isOutputPointsIn[15] and isOutputPointsIn[16]:
        ax2.plot([output_x[15],output_x[16]], [output_y[15],output_y[16]], [output_z[15],output_z[16]], zdir='y', c='#0066ff')
    if isOutputPointsIn[0] and isOutputPointsIn[17]:
        ax2.plot([output_x[0],output_x[17]], [output_y[0],output_y[17]], [output_z[0],output_z[17]], zdir='y', c='#a300cc')
    if isOutputPointsIn[17] and isOutputPointsIn[18]:
        ax2.plot([output_x[17],output_x[18]], [output_y[17],output_y[18]], [output_z[17],output_z[18]], zdir='y', c='#8f00b3')
    if isOutputPointsIn[18] and isOutputPointsIn[19]:
        ax2.plot([output_x[18],output_x[19]], [output_y[18],output_y[19]], [output_z[18],output_z[19]], zdir='y', c='#b800e6')
    if isOutputPointsIn[19] and isOutputPointsIn[20]:
        ax2.plot([output_x[19],output_x[20]], [output_y[19],output_y[20]], [output_z[19],output_z[20]], zdir='y', c='#cc00ff')
    #ax.scatter(input_x, input_y, c='red', label = 'input')
    #ax.scatter(output_x, output_y, c='blue', label = 'output')
    
    #for l in range(len(input_x)):
    #    connect_x = [input_x[l], output_x[l]]
    #    connect_y = [input_y[l], output_y[l]]
    #    connect_z = [input_z[l], output_z[l]]
    #    ax.plot(connect_x,connect_y,connect_z,zdir='y',color='gray')
    ###stradrs = str(train_adrs[target_epoch][i].item()) #202011221813290002
    #print(stradrs[:8])
    #print(stradrs[8:14])
    #print(stradrs[14:])
    ###d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ###ax.set_title('output(train data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax2.set_title("decoder test")
    ax2.set_xlim([x_min, x_max])
    ax2.set_ylim([z_min, z_max])
    ax2.set_zlim([y_min, y_max])
    ax2.set_xlabel('x')
    ax2.set_ylabel('z')
    ax2.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    ###fig.savefig(PATH + "\\Output_train_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300,transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig2.savefig(PATH + "\\decoder_test.png", dpi = 300,transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig2.show()
    fig2.clf()

[-0.026983162388205528, -1.6507606953382492, -4.9086302518844604, -9.228235483169556, -10.960614681243896, -14.788538217544556, -21.92721664905548, -25.97249448299408, -28.263840079307556, -17.168140411376953, -25.308623909950256, -29.49046492576599, -32.15322494506836, -17.90042221546173, -25.479114055633545, -29.345837235450745, -31.732308864593506, -17.649246752262115, -23.37656319141388, -26.16560161113739, -27.570414543151855]
[-0.020072259940207005, -0.8222064934670925, -1.9345350563526154, -4.560711607336998, -7.986330986022949, 6.09380342066288, 1.8010500818490982, -0.9707370772957802, -3.585543856024742, 4.999532550573349, 0.40018167346715927, -3.1514912843704224, -5.57241290807724, 1.4698466286063194, -2.7706611901521683, -6.340382248163223, -9.090659022331238, -2.805967628955841, -6.667006015777588, -9.058225154876709, -10.957257449626923]
[-0.030663050711154938, 2.3051440715789795, 7.854856550693512, 10.071006417274475, 14.120686054229736, 8.985626697540283, 15.419337153434

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py:457: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py:457: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py:457: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py:457: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py:457: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure

<Figure size 432x288 with 0 Axes>